In [61]:
from initial_functions import fn_read_from_db, fn_write_to_db,fn_max_date_calc
import pandas as pd


ImportError: cannot import name 'fn_max_date_calc' from 'initial_functions' (/Users/irfanakgul/Desktop/TradeAI-APP/src/initial_functions.py)

In [6]:
df_raw = fn_read_from_db('tbl_ticker_actual_daily')

In [49]:
df_t1 = df_raw[df_raw['TICKER']=='ULKER.IS']

In [51]:
list(df_t1['DATETIME'])

['2024-01-02 00:00:00+03:00',
 '2024-01-03 00:00:00+03:00',
 '2024-01-04 00:00:00+03:00',
 '2024-01-05 00:00:00+03:00',
 '2024-01-08 00:00:00+03:00',
 '2024-01-09 00:00:00+03:00',
 '2024-01-10 00:00:00+03:00',
 '2024-01-11 00:00:00+03:00',
 '2024-01-12 00:00:00+03:00',
 '2024-01-15 00:00:00+03:00',
 '2024-01-16 00:00:00+03:00',
 '2024-01-17 00:00:00+03:00',
 '2024-01-18 00:00:00+03:00',
 '2024-01-19 00:00:00+03:00',
 '2024-01-22 00:00:00+03:00',
 '2024-01-23 00:00:00+03:00',
 '2024-01-24 00:00:00+03:00',
 '2024-01-25 00:00:00+03:00',
 '2024-01-26 00:00:00+03:00',
 '2024-01-29 00:00:00+03:00',
 '2024-01-30 00:00:00+03:00',
 '2024-01-31 00:00:00+03:00',
 '2024-02-01 00:00:00+03:00',
 '2024-02-02 00:00:00+03:00',
 '2024-02-05 00:00:00+03:00',
 '2024-02-06 00:00:00+03:00',
 '2024-02-07 00:00:00+03:00',
 '2024-02-08 00:00:00+03:00',
 '2024-02-09 00:00:00+03:00',
 '2024-02-12 00:00:00+03:00',
 '2024-02-13 00:00:00+03:00',
 '2024-02-14 00:00:00+03:00',
 '2024-02-15 00:00:00+03:00',
 '2024-02-

In [32]:
import pandas as pd

def calculate_range_metrics(df, range, TICKER, CUTT_OFF=None):
    
    # Orijinal df’i bozmamak için kopya al
    df_work = df.copy()

    # Sadece sıralama (DATETIME'e dokunmuyoruz)
    df_work = df_work.sort_values('DATETIME', ascending=False).reset_index(drop=True)

    # Son tarih (orijinal haliyle)
    end_datetime = df_work.iloc[0]['DATETIME']
    print(end_datetime)

    # Range mapping
    range_map = {
        "1day": pd.DateOffset(days=1),
        "1week": pd.DateOffset(weeks=1),
        "1months": pd.DateOffset(months=1),
        "3months": pd.DateOffset(months=3),
        "6months": pd.DateOffset(months=6),
        "1year": pd.DateOffset(years=1),
        "2year": pd.DateOffset(years=2)
    }

    if range not in range_map:
        raise ValueError("Geçersiz range değeri")

    # RANGE_STARTDATE (timezone korunur)
    range_startdate = end_datetime - range_map[range]

    # RANGE filtre
    df_filter = df_work[
        (df_work['DATETIME'] >= range_startdate) &
        (df_work['DATETIME'] <= end_datetime)
    ].copy()

    range_size = len(df_filter)

    # Highest
    highest_idx = df_filter['HIGH'].idxmax()
    highest_value = round(df_filter.loc[highest_idx, 'HIGH'], 2)
    highest_date = df_filter.loc[highest_idx, 'DATETIME']  # dokunmadan alıyoruz

    # Highest sonrası filtre
    if CUTT_OFF is not None:
        df_after = df_filter[
            (df_filter['DATETIME'] >= highest_date) &
            (df_filter['DATETIME'] <= CUTT_OFF)
        ]
    else:
        df_after = df_filter[df_filter['DATETIME'] >= highest_date]

    row_size_after_highest = len(df_after)

    end_datetime_filtered = df_filter['DATETIME'].max()

    df_res = pd.DataFrame([{
        "RANGE_STARTDATE": range_startdate,
        "RANGE_TYPE": range,
        "TICKER": TICKER,
        "RANGE_SIZE": range_size,
        "HIGHEST_VALUE": highest_value,
        "HIGHEST_DATE": highest_date,
        "ROW_SIZE_AFTER_HIGHEST": row_size_after_highest,
        "END_DATETIME": end_datetime_filtered
    }])

    return df_res



In [34]:
df_res = calculate_range_metrics(df_t1, '6months', 'ULKER.IS', CUTT_OFF=None)

2026-02-09 00:00:00+03:00


In [30]:
df_res

,RANGE_STARTDATE,RANGE_TYPE,TICKER,RANGE_SIZE,HIGHEST_VALUE,HIGHEST_DATE,ROW_SIZE_AFTER_HIGHEST,END_DATETIME
0,2025-08-09 00:00:00+03:00,6months,ULKER.IS,129,139.5,2026-02-04 00:00:00+03:00,4,2026-02-09 00:00:00+03:00
